In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime
import json

# Initialize Spark Session (without Delta configurations)
spark = SparkSession.builder \
    .appName("F1SilverLayer") \
    .getOrCreate()

# First, let's read the parquet files and see what we're working with
def process_f1_silver_layer():
    try:
        print("Starting Silver Layer Processing...")

        # Read the parquet files
        bronze_df = spark.read.json(
            "/content/drive/MyDrive/Capstone/bronze/season=*"
        )

        print("Bronze data loaded. Starting transformations...")

        # 1. Explode nested structures
        df = bronze_df.select(
            F.col("season"),
            F.col("round"),
            F.col("raceName"),
            F.col("date"),
            F.col("time"),
            F.col("url"),
            F.col("Circuit").alias("circuit"),
            F.explode("Results").alias("result")
        )

        # 2. Flatten nested structures
        df = df.select(
            "*",
            F.col("circuit.circuitId").alias("circuit_id"),
            F.col("circuit.circuitName").alias("circuit_name"),
            F.col("circuit.Location.lat").alias("circuit_lat"),
            F.col("circuit.Location.long").alias("circuit_long"),
            F.col("circuit.Location.locality").alias("circuit_locality"),
            F.col("circuit.Location.country").alias("circuit_country"),
            F.col("result.Constructor.constructorId").alias("constructor_id"),
            F.col("result.Constructor.name").alias("constructor_name"),
            F.col("result.Driver.driverId").alias("driver_id"),
            F.col("result.Driver.givenName").alias("driver_given_name"),
            F.col("result.Driver.familyName").alias("driver_family_name"),
            F.col("result.position").alias("position"),
            F.col("result.points").alias("points"),
            F.col("result.grid").alias("grid"),
            F.col("result.laps").alias("laps"),
            F.col("result.status").alias("status"),
            F.col("result.Time.time").alias("finish_time")
        ).drop("circuit", "result")

        # 3. Data type conversions and standardization
        df = df.withColumn("race_timestamp",
                          F.to_timestamp(
                              F.concat(F.col("date"), F.lit(" "), F.col("time")),
                              "yyyy-MM-dd HH:mm:ssX"
                          ))

        # 4. Add computed columns
        df = df.withColumn("driver_full_name",
                          F.concat(F.col("driver_given_name"),
                                 F.lit(" "),
                                 F.col("driver_family_name")))

        # 5. Data quality checks
        df = df.withColumn("is_valid_position",
                          (F.col("position").isNotNull() &
                           F.col("position").cast("int").isNotNull() &
                           (F.col("position").cast("int") >= 1)))

        df = df.withColumn("is_valid_points",
                          (F.col("points").isNotNull() &
                           F.col("points").cast("double").isNotNull() &
                           (F.col("points").cast("double") >= 0)))

        df = df.withColumn("is_valid_grid",
                          (F.col("grid").isNotNull() &
                           F.col("grid").cast("int").isNotNull() &
                           (F.col("grid").cast("int") >= 0)))

        df = df.withColumn("is_valid_date",
                          F.col("race_timestamp").isNotNull())

        # Combine all checks
        df = df.withColumn("is_valid_record",
                          F.col("is_valid_position") &
                          F.col("is_valid_points") &
                          F.col("is_valid_grid") &
                          F.col("is_valid_date"))

        # 6. Add metadata
        df = df.withColumn("processed_timestamp", F.current_timestamp())

        # 7. Calculate quality metrics
        total_records = df.count()
        valid_records = df.filter(F.col("is_valid_record")).count()
        null_percentages = {}

        for column in df.columns:
            null_count = df.filter(F.col(column).isNull()).count()
            null_percentages[column] = (null_count / total_records) * 100

        print("\nData Quality Metrics:")
        print(f"Total Records: {total_records}")
        print(f"Valid Records: {valid_records}")
        print(f"Invalid Records: {total_records - valid_records}")
        print("\nNull Percentages:")
        for col, pct in null_percentages.items():
            if pct > 0:
                print(f"{col}: {pct:.2f}%")

        # 8. Write to silver layer
        print("\nWriting to silver layer...")

        # Write main dataset
        silver_path = "/content/drive/MyDrive/Capstone/silver1"
        df.write.mode("overwrite") \
            .partitionBy("season") \
            .parquet(silver_path)

        print(f"\nSilver layer data written to: {silver_path}")
        print("Silver Layer Processing Complete!")

        return df

    except Exception as e:
        print(f"Error in Silver Layer Processing: {str(e)}")
        raise

# Execute the processing
silver_df = process_f1_silver_layer()

# Show sample of the processed data
print("\nSample of processed data:")
silver_df.show(5)

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime
import json

# Initialize Spark Session (without Delta configurations)
spark = SparkSession.builder \
    .appName("F1SilverLayer") \
    .getOrCreate()

bronze_df = spark.read.json(
            "/content/drive/MyDrive/Capstone/bronze/season=*"
        )

In [ ]:
bronze_df.show()

+--------------------+--------------------+----------+--------------------+--------------------+-----+------+-----------+---------+--------------------+
|             Circuit|             Results|      date| ingestion_timestamp|            raceName|round|season|     source|     time|                 url|
+--------------------+--------------------+----------+--------------------+--------------------+-----+------+-----------+---------+--------------------+
|{{Bahrain, 26.032...|[{{red_bull, Red ...|2024-03-02|2025-04-16T07:19:...|  Bahrain Grand Prix|    1|  2024|jolpica_api|15:00:00Z|https://en.wikipe...|
|{{Saudi Arabia, 2...|[{{red_bull, Red ...|2024-03-09|2025-04-16T07:19:...|Saudi Arabian Gra...|    2|  2024|jolpica_api|17:00:00Z|https://en.wikipe...|
|{{Australia, -37....|[{{ferrari, Ferra...|2024-03-24|2025-04-16T07:19:...|Australian Grand ...|    3|  2024|jolpica_api|04:00:00Z|https://en.wikipe...|
|{{Japan, 34.8431,...|[{{red_bull, Red ...|2024-04-07|2025-04-16T07:19:...| Japane

In [ ]:
bronze_df.repartition(1).write.parquet('/content/drive/MyDrive/Capstone/tmp')

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, expr, lit

# Initialize Spark session with appropriate configurations for Jupyter
spark = SparkSession.builder \
    .appName("F1 Data Transformation") \
    .config("spark.driver.memory", "4g") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.host", "localhost") \
    .master("local[*]") \
    .getOrCreate()

# File paths - update these to your actual file locations
output_path = "/content/drive/MyDrive/Capstone/tmp"

# OPTION 1: If PySpark can directly read the parquet file
try:
    # Try direct reading with PySpark
    df_parquet = spark.read.json("/content/drive/MyDrive/Capstone/bronze/season=*")
    print("Successfully read parquet file directly with PySpark")
except Exception as e:
    print(f"Direct PySpark read failed: {e}")
    print("Falling back to pandas + pyarrow method")


# Display schema and sample data
print("\\nParquet file schema:")
df_parquet.printSchema()

print("\\nSample data from parquet file:")
df_parquet.show(2, truncate=False)

# Explode the Results array to create one row per driver result
df_exploded = df_parquet.select(
    col("season"),
    col("round"),
    col("url").alias("race_url"),
    col("raceName"),
    col("date"),
    col("time"),
    col("Circuit.circuitId").alias("circuitId"),
    col("Circuit.url").alias("circuit_url"),
    col("Circuit.circuitName").alias("circuitName"),
    col("Circuit.Location.lat").alias("circuit_lat"),
    col("Circuit.Location.long").alias("circuit_long"),
    col("Circuit.Location.locality").alias("circuit_locality"),
    col("Circuit.Location.country").alias("circuit_country"),
    explode(col("Results")).alias("result")
)

# Extract fields from the exploded result
df_flattened = df_exploded.select(
    col("season"),
    col("round"),
    col("race_url"),
    col("raceName"),
    col("date"),
    col("time"),
    col("circuitId"),
    col("circuit_url"),
    col("circuitName"),
    col("circuit_lat"),
    col("circuit_long"),
    col("circuit_locality"),
    col("circuit_country"),
    col("result.number").alias("driverNumber"),
    col("result.position").alias("position"),
    col("result.positionText").alias("positionText"),
    col("result.points").alias("points"),
    col("result.grid").alias("grid"),
    col("result.laps").alias("laps"),
    col("result.status").alias("status"),
    col("result.Time.time").alias("raceTime"),
    col("result.Time.millis").alias("raceTimeMillis"),
    col("result.Driver.driverId").alias("driverId"),
    col("result.Driver.permanentNumber").alias("permanentNumber"),
    col("result.Driver.code").alias("code"),
    col("result.Driver.url").alias("driver_url"),
    col("result.Driver.givenName").alias("givenName"),
    col("result.Driver.familyName").alias("familyName"),
    col("result.Driver.dateOfBirth").alias("dateOfBirth"),
    col("result.Driver.nationality").alias("driver_nationality"),
    col("result.Constructor.constructorId").alias("constructorId"),
    col("result.Constructor.url").alias("constructor_url"),
    col("result.Constructor.name").alias("constructorName"),
    col("result.Constructor.nationality").alias("constructor_nationality"),
    col("result.FastestLap.rank").alias("fastestLapRank"),
    col("result.FastestLap.lap").alias("fastestLapNumber"),
    col("result.FastestLap.Time.time").alias("fastestLapTime"),
    col("result.FastestLap.AverageSpeed.speed").alias("fastestLapAvgSpeed"),
    col("result.FastestLap.AverageSpeed.units").alias("fastestLapSpeedUnits")
)

# Show the flattened data
print("\\nFlattened data schema:")
df_flattened.printSchema()

print("\\nFlattened data sample (first 2 rows):")
df_flattened.show(2, truncate=False)

# Count the number of rows
row_count = df_flattened.count()
print(f"\\nTotal number of rows in flattened data: {row_count}")



df_flattened.repartition(1).write.mode("overwrite").option("header", "true").csv(f"{output_path}/csv")



Successfully read parquet file directly with PySpark
\nParquet file schema:
root
 |-- Circuit: struct (nullable = true)
 |    |-- Location: struct (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- lat: string (nullable = true)
 |    |    |-- locality: string (nullable = true)
 |    |    |-- long: string (nullable = true)
 |    |-- circuitId: string (nullable = true)
 |    |-- circuitName: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- Results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Constructor: struct (nullable = true)
 |    |    |    |-- constructorId: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- nationality: string (nullable = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |-- Driver: struct (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- dateOfBirth: string (nullable = tr

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Capstone/tmp/csv/part-00000-58b152f1-76d0-471d-9df3-4ca5b5b318f8-c000.csv',low_memory=False)

In [ ]:
df.head()

,season,round,race_url,raceName,date,time,circuitId,circuit_url,circuitName,circuit_lat,...,driver_nationality,constructorId,constructor_url,constructorName,constructor_nationality,fastestLapRank,fastestLapNumber,fastestLapTime,fastestLapAvgSpeed,fastestLapSpeedUnits
0,2024,1,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,Bahrain Grand Prix,2024-03-02,15:00:00Z,bahrain,https://en.wikipedia.org/wiki/Bahrain_Internat...,Bahrain International Circuit,26.0325,...,Dutch,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian,1.0,39.0,1:32.608,210.383,kph
1,2024,1,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,Bahrain Grand Prix,2024-03-02,15:00:00Z,bahrain,https://en.wikipedia.org/wiki/Bahrain_Internat...,Bahrain International Circuit,26.0325,...,Mexican,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian,4.0,40.0,1:34.364,206.468,kph
2,2024,1,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,Bahrain Grand Prix,2024-03-02,15:00:00Z,bahrain,https://en.wikipedia.org/wiki/Bahrain_Internat...,Bahrain International Circuit,26.0325,...,Spanish,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,6.0,44.0,1:34.507,206.156,kph
3,2024,1,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,Bahrain Grand Prix,2024-03-02,15:00:00Z,bahrain,https://en.wikipedia.org/wiki/Bahrain_Internat...,Bahrain International Circuit,26.0325,...,Monegasque,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,2.0,36.0,1:34.090,207.069,kph
4,2024,1,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,Bahrain Grand Prix,2024-03-02,15:00:00Z,bahrain,https://en.wikipedia.org/wiki/Bahrain_Internat...,Bahrain International Circuit,26.0325,...,British,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German,12.0,40.0,1:35.065,204.946,kph
